In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- 1. Load and Prepare the Data ---
df = pd.read_excel('Final_With_HV.xlsx')
X = df.drop('Option_Price_C', axis=1)
y = df['Option_Price_C']

# --- 2. Split and Scale the Data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 3. Build and Compile the Model ---
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# --- CHANGE: Added RootMeanSquaredError to the metrics ---
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=[tf.keras.metrics.RootMeanSquaredError()] # Monitor RMSE during training
)

# --- ADD THIS PART ---
# Create the learning rate scheduler callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Monitor the validation loss
    factor=0.2,          # Reduce the learning rate by a factor of 0.2 (e.g., 0.001 -> 0.0002)
    patience=5,          # Reduce if the metric doesn't improve for 5 consecutive epochs
    min_lr=0.000005,      # The lowest the learning rate is allowed to go
    verbose=1            # Print a message when the learning rate is updated
)
model.summary()

# --- 4. Train the Model ---
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    validation_split=0.2,
    callbacks=[lr_scheduler], # Add the callback here
    verbose=1 # Set to 0 to keep the output clean
)
print("Model training finished.")

# --- ADD THIS LINE TO SAVE YOUR MODEL ---
model.save('option_price_predictor_with_hv.keras')

print("Model saved successfully to 'option_price_predictor_with_hv.keras'")
# --- 5. Evaluate and Predict ---

# --- CHANGE: Evaluate and display the RMSE ---
loss, rmse = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nTest Set Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"This means the model's predictions are, on average, off by about ${rmse:.2f}.")

# Make predictions and show a sample comparison
predictions = model.predict(X_test_scaled).flatten()
results = pd.DataFrame({'Actual_Price': y_test, 'Predicted_Price': predictions})
print("\nSample Predictions:")
print(results.head())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,521 (45.00 KB)

 Trainable params: 11,521 (45.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 171908.2188 - root_mean_squared_error: 388.3975 - val_loss: 582.9741 - val_root_mean_squared_error: 24.1449 - learning_rate: 0.0010
Epoch 2/100
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 470.9264 - root_mean_squared_error: 21.6374 - val_loss: 310.0477 - val_root_mean_squared_error: 17.6082 - learning_rate: 0.0010
Epoch 3/100
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 233.6824 - root_mean_squared_error: 15.2813 - val_loss: 311.9268 - val_root_mean_squared_error: 17.6614 - learning_rate: 0.0010
Epoch 4/100
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 160.6869 - root_mean_squared_error: 12.6732 - val_loss: 102.6999 - val_root_mean_squared_error: 10.1341 - learning_rate: 0.0010
Epoch 5/100
3063/3063 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 114.7924 - root_mean_squared_error: 10.7092 - val_loss: 150.9632 - val_root_mean_squared_error: 12.2867 - learning_rate: 0.0010
Epoch 6/100
3063/3063 ━━━━━━━━━━━━━━━━━━